# Example 26: Policy Iteration

## Contents
* [Acknowledgements](#ackw)
* [Overview](#overview) 
    * [Policy iteration algorithm](#ekf)
    * [Test case](#motion_model)
* [Include files](#include_files)
* [The main function](#m_func)
* [Results](#results)
* [Source Code](#source_code)

## <a name="ackw"></a> Acknowledgements

Most of the content in this notebook is taken from the book ```Reinforcement Learning An introduction``` by Sutton and Barto. Note that the C++ code is an adaptation of the Python code in the repository <a href="https://github.com/ShangtongZhang/reinforcement-learning-an-introduction">ShangtongZhang/reinforcement-learning-an-introduction</a>.

## <a name="overview"></a> Overview

In this notebook we will discuss the so-called Policy Iteration algorithm for a finite MDP. 

### <a name="ekf"></a> Policy iteration algorithm

The idea behind policy iteration is rather simple; once a policy $\pi$ has been improved using $V_{\pi}$ to yield a better policy say $\pi_1$ we can then compute $V_{\pi_1}$ and improve it again to yield an even better policy say $\pi_2$. This is of course true provided that the used policy is not optimal already. This process will give us a sequence of monotonically improving policies and value fuctions:


$$\pi_0 \rightarrow V_{\pi_0} \rightarrow \pi_1 \rightarrow V_{\pi_1}\rightarrow \pi_2 \cdots \pi_{*} \rightarrow V_{\pi_{*}}$$

The step $\pi_i \rightarrow V_{\pi_i}$ is an evaluation step. On the other hand the step $V_{\pi_i} \rightarrow \pi_{i+1} $ is an improvement step. We know that we can use iterative policy evaluation at this step. Each policy is guaranteed to be a strict improvement over the previous one (unless it is already optimal).

Since we are assuming a finite MDP, this means that  only a finite number of policies exists. Thus, this process must converge to an optimal policy and optimal value function in a finite number of iterations. This way of finding an optimal policy is called **policy iteration** (see ```Reinforcement Learning An introduction``` by Sutton and Barton).  

Note that each policy evaluation is itself an iterative computation. It
is started with the value function from the previous policy. This most of the times results in a great
increase in the speed of convergence of policy evaluation.

### <a name="motion_model"></a> Test case

The test case we will consider is the own described in the relevant section of the book ```Reinforcement Learning An introduction``` by Sutton and Barto. Concretely, the test case  is example 4.2. The test case is as follows: 

Jack manages two locations, denoted by $L_1$ and $L_2$ respectively, for a nationwide car rental company. Each day, some number of customers arrive at each location to rent cars.  If Jack has a car available, herents it out and is credited \\$10 by the national company.  If he is out of cars at that location, then thebusiness is lost.  Cars become available for renting the day after they are returned.  To help ensure thatcars are available where they are needed, Jack can move them between the two locations overnight, ata cost of \\$2 per car moved.  We assume that the number of cars requested and returned at each location are Poisson random variables, meaning that the probability that the number is

$$ \frac{\lambda^n}{n!}e^{-\lambda}$$

We assume that there can be no more than 20 cars at each location (any additional cars are returned to the nationwide company, and thus disappear from the problem) and a maximum of five cars can be moved from one location to the other in onenight.

Let $S=(S_1,S_2)$ be the tuple that denotes the number of cars at locations $L_1$ and $L_2$. The action set is 

$$A=\{0,1,2,3,4,5\}$$

The discount rate is set to  $\gamma = 0.9$.

## <a name="include_files"></a> Include files

```
#include "cubic_engine/base/cubic_engine_types.h"
#include "kernel/utilities/csv_file_writer.h"
#include "kernel/base/kernel_consts.h"
#include "kernel/utilities/array_utils.h"

#include <cmath>
#include <utility>
#include <tuple>
#include <iostream>
#include <random>
#include <algorithm>
#include <cmath>
#include <limits>
#include <map>
```

## <a name="m_func"></a> The main function

```
namespace example
{
using cengine::uint_t;
using cengine::real_t;
using cengine::DynMat;
using kernel::CSVWriter;

// max number of cars in each location
const int MAX_CARS = 20;

// max number of cars to move during night
const uint_t MAX_MOVE_OF_CARS = 5;

// expectation for rental requests in first location
const uint_t RENTAL_REQUEST_FIRST_LOC = 3;

// expectation for rental requests in second location
const uint_t RENTAL_REQUEST_SECOND_LOC = 4;

// expectation for # of cars returned in first location
const int RETURNS_FIRST_LOC = 3;

// expectation for # of cars returned in second location
const int RETURNS_SECOND_LOC = 2;

// discount factor
const real_t DISCOUNT = 0.9;

// credit earned by a car
const real_t RENTAL_CREDIT = 10;

// cost of moving a car
const real_t MOVE_CAR_COST = 2;

// all possible actions
int actions[] = {-5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5};

// An up bound for poisson distribution
// If n is greater than this value, then the probability
// of getting n is truncated to 0
const uint_t POISSON_UPPER_BOUND = 11;

// cache for the Poisson values
std::map<int, real_t> poisson_cache;

real_t poisson_pmf(int  k, real_t lambda) {
    return std::exp(k * std::log(lambda) - std::lgamma(k + 1.0) - lambda);
  }

real_t poisson_probability(int n, int lam ){

    auto key = n * 10 + lam;
    auto itr = poisson_cache.find(key);
    if(itr == poisson_cache.end()){
        poisson_cache[key] = poisson_pmf(n, lam);
    }

    return poisson_cache[key];
}

real_t expected_return(int state[], int action, DynMat<real_t>& state_value, bool constant_returned_cars){

    // initailize total return
    real_t returns = 0.0;

    // cost for moving cars
    returns -= MOVE_CAR_COST * std::abs(action);

    // moving cars
    int NUM_OF_CARS_FIRST_LOC = std::min(state[0] - action, MAX_CARS);
    int NUM_OF_CARS_SECOND_LOC = std::min(state[1] + action, MAX_CARS);

    // go through all possible rental requests
    for(int rental_request_first_loc=0;  rental_request_first_loc<POISSON_UPPER_BOUND; ++rental_request_first_loc){
        for(int rental_request_second_loc=0;  rental_request_second_loc<POISSON_UPPER_BOUND; ++rental_request_second_loc){

            // probability for current combination of rental requests
            auto prob = poisson_probability(rental_request_first_loc, RENTAL_REQUEST_FIRST_LOC);
            prob *= poisson_probability(rental_request_second_loc, RENTAL_REQUEST_SECOND_LOC);

            auto num_of_cars_first_loc = NUM_OF_CARS_FIRST_LOC;
            auto num_of_cars_second_loc = NUM_OF_CARS_SECOND_LOC;

            // valid rental requests should be less than actual # of cars
            auto valid_rental_first_loc = std::min(num_of_cars_first_loc, rental_request_first_loc);
            auto valid_rental_second_loc = std::min(num_of_cars_second_loc, rental_request_second_loc);

            // get credits for renting
            auto reward = (valid_rental_first_loc + valid_rental_second_loc) * RENTAL_CREDIT;
            num_of_cars_first_loc -= valid_rental_first_loc;
            num_of_cars_second_loc -= valid_rental_second_loc;

            if( constant_returned_cars){
                // get returned cars, those cars can be used for renting tomorrow
                auto returned_cars_first_loc = RETURNS_FIRST_LOC;
                auto returned_cars_second_loc = RETURNS_SECOND_LOC;
                num_of_cars_first_loc = std::min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS);
                num_of_cars_second_loc = std::min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS);
                returns += prob * (reward + DISCOUNT * state_value(num_of_cars_first_loc, num_of_cars_second_loc));
            }
            else{
                for(int returned_cars_first_loc=0; returned_cars_first_loc<POISSON_UPPER_BOUND; ++returned_cars_first_loc){
                    for( int returned_cars_second_loc=0; returned_cars_second_loc<POISSON_UPPER_BOUND; ++returned_cars_second_loc){

                        auto prob_return = poisson_probability(
                            returned_cars_first_loc, RETURNS_FIRST_LOC) * poisson_probability(returned_cars_second_loc, RETURNS_SECOND_LOC);
                        auto num_of_cars_first_loc_ = std::min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS);
                        auto num_of_cars_second_loc_ = std::min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS);
                        auto prob_ = prob_return * prob;
                        returns += prob_ * (reward + DISCOUNT *
                                            state_value(num_of_cars_first_loc_, num_of_cars_second_loc_));
                    }
                }
           }
          }
         }
    return returns;

}


void simulate(bool constant_returned_cars){

    DynMat<real_t> value(MAX_CARS + 1, MAX_CARS + 1, 0.);
    DynMat<int> policy(MAX_CARS + 1, MAX_CARS + 1, -1);

    // how many iteration have we performed
    uint_t iterations = 0;

    while(true){

        // policy evaluation (in-place)
        while(true) {

           // a copy of the current value function
           auto old_value = value;

           for(int i=0; i<(MAX_CARS + 1); ++i){
               for(int j=0; j<(MAX_CARS + 1); ++j){

                   int local_state[] = {i,j};
                   auto new_state_value = expected_return(local_state, policy(i,j),
                                                          value, constant_returned_cars);
                   value(i,j) = new_state_value;
               }

               auto  max_value_change = max(abs(old_value - value));
               std::cout<<"max value change: "<<max_value_change<<std::endl;
               if(max_value_change < 1e-4){
                    break;
               }
           }
        }

        // policy improvement
        bool policy_stable = true;

        for(int i=0; i<(MAX_CARS + 1); ++i){
            for(int j=0; j<(MAX_CARS + 1); ++j){

                auto old_action = policy(i, j);
                std::vector<real_t> action_returns;

                for(uint_t a=0; a<11; ++a){

                    auto action = actions[a];
                    if((0 <= action <= i) || (-j <= action <= 0)){
                        int local_state[] = {i,j};
                        action_returns.push_back(expected_return(local_state, action,
                                                                 value, constant_returned_cars));
                    }
                    else{
                        action_returns.push_back(std::numeric_limits<real_t>::min());
                    }
                }

                auto new_action = actions[kernel::utils::arg_max(action_returns)];
                policy(i, j) = new_action;
                if( policy_stable && old_action != new_action){
                        policy_stable = false;
                }
            }
        }

        if(policy_stable){
            break;
        }
}
}


}

int main(){


    using namespace example;


    try{
        simulate(true);
    }
    catch(std::exception& e){

        std::cerr<<e.what()<<std::endl;
    }
    catch(...){

        std::cerr<<"Unknown exception occured"<<std::endl;
    }

    return 0;
}


```

## <a name="results"></a> Results



```
```

The following images shown the sum of rewards achieved by the algorithm for various values of $\epsilon$. We see that as $\epsilon$ is increased more exploration occurs.

## <a name="source_code"></a> Source Code



<a href="../exe.cpp">exe.cpp</a>